In [ ]:
from pathlib import Path
import fitz  # PyMuPDF
import os
from dotenv import load_dotenv

from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


In [ ]:
# ✅ Load environment variables
load_dotenv()
persist_dir = "db"
pdf_dir = Path("data/tutorials")  # Adjust path to your actual PDF folder

/var/folders/cq/tvx_t3ws7xxcsv5lz1_j4j_40000gn/T/ipykernel_10289/4034385780.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)


In [ ]:
# ✅ Set up embedding model using HuggingFace
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# ✅ Extract text from a single PDF
def extract_text_from_pdf(pdf_path):
    print(f"📄 Reading: {pdf_path.name}")
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        print(f"❌ Failed to read {pdf_path.name}: {e}")
        return ""

FileNotFoundError: ❌ No PDF files found in /data folder.

In [ ]:
# ✅ Load all PDFs
pdf_files = list(pdf_dir.glob("*.pdf"))
if not pdf_files:
    raise FileNotFoundError("❌ No PDF files found in 'data/tutorials/'")

all_text = ""
for file in pdf_files:
    all_text += extract_text_from_pdf(file)

print("✅ All PDF content loaded.")

In [ ]:
# ✅ Chunk the text
splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_text(all_text)
print(f"✅ Total chunks created: {len(chunks)}")

In [ ]:
# ✅ Embed and store in Chroma
db = Chroma.from_texts(chunks, embedding, persist_directory=persist_dir)
db.persist()
print(f"✅ Vector database saved to '{persist_dir}'")